#### Compare Models

In [8]:
import sys
!{sys.executable} -m pip install numpy pandas matplotlib scikit-learn seaborn imblearn | grep -v 'already satisfied'

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sb

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
%matplotlib inline 
SEED = 618


Defaulting to user installation because normal site-packages is not writeable
distutils: /home/re34/.local/lib/python3.9/site-packages
sysconfig: /home/re34/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


In [9]:
# Read in data
train = pd.read_csv("/cs/studres/ID5059/Coursework/Coursework-2/data/train.csv")
test = pd.read_csv("/cs/studres/ID5059/Coursework/Coursework-2/data/test.csv")

# Clear the maximum number of columns to be displayed, so that all will be visible.
pd.set_option('display.max_columns', None)

# set id as index
train = train.set_index('id')
test = test.set_index('id')

# split train set into X_train, y_train
y_train = train['Class']
X_train = train.drop('Class', axis = 1)

train.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,2.074329,-0.129425,-1.137418,0.412846,-0.192638,-1.210144,0.110697,-0.263477,0.742144,0.108782,-1.070243,-0.234910,-1.099360,0.502467,0.169318,0.065688,-0.306957,-0.323800,0.103348,-0.292969,-0.334701,-0.887840,0.336701,-0.110835,-0.291459,0.207733,-0.076576,-0.059577,1.98,0
1,0.0,1.998827,-1.250891,-0.520969,-0.894539,-1.122528,-0.270866,-1.029289,0.050198,-0.109948,0.908773,0.836798,-0.056580,-0.120990,-0.144028,-0.039582,1.653057,-0.253599,-0.814354,0.716784,0.065717,0.054848,-0.038367,0.133518,-0.461928,-0.465491,-0.464655,-0.009413,-0.038238,84.00,0
2,0.0,0.091535,1.004517,-0.223445,-0.435249,0.667548,-0.988351,0.948146,-0.084789,-0.042027,-0.818383,-0.376512,-0.226546,-0.552869,-0.886466,-0.180890,0.230286,0.590579,-0.321590,-0.433959,-0.021375,-0.326725,-0.803736,0.154495,0.951233,-0.506919,0.085046,0.224458,0.087356,2.69,0
3,0.0,1.979649,-0.184949,-1.064206,0.120125,-0.215238,-0.648829,-0.087826,-0.035367,0.885838,-0.007527,0.637441,0.676960,-1.504823,0.554039,-0.824356,-0.527267,-0.095838,-0.312519,0.642659,-0.340089,-0.095514,-0.079792,0.167701,-0.042939,0.000799,-0.096148,-0.057780,-0.073839,1.00,0
4,0.0,1.025898,-0.171827,1.203717,1.243900,-0.636572,1.099074,-0.938651,0.569239,0.692665,-0.097495,1.338869,1.391399,-0.128167,-0.081836,0.100548,-0.338937,0.090864,-0.423645,-0.731939,-0.203628,0.099157,0.608908,0.027901,-0.262813,0.257834,-0.252829,0.108338,0.021051,1.00,0


- decision tree with ada boost

In [11]:
decision_tree_simple = DecisionTreeClassifier(max_depth = 1, random_state = SEED);
decision_tree_simple.fit(X_train, y_train)

ada_final = AdaBoostClassifier(decision_tree_simple, n_estimators = 150, algorithm = "SAMME.R", learning_rate = 0.1, random_state = SEED);
trained_ada_boost= ada_final.fit(X_train, y_train)

- logistic regression model Elastic

In [13]:
# Scale the training data
scaler_elastic = StandardScaler()
X_elastic_scaled = scaler_elastic.fit_transform(X_train)

# define logisitc regressor with elastic net
logi_elastic_train = LogisticRegression(C = 0.1, l1_ratio= 0.9, penalty='elasticnet', solver='saga', max_iter=500)
trained_logistic_elastic = logi_elastic_train.fit(X_elastic_scaled, y_train)

- logistic regression model SMOTE

In [15]:
# apply SMOTE to oversample the minority class on the original data
smote = SMOTE(sampling_strategy='minority')
X_train_SMOTE, y_train_SMOTE = smote.fit_resample(X_train, y_train)

# apply random undersampling to balance the class distribution
rus = RandomUnderSampler(sampling_strategy='majority')
X_train_SMOTE, y_train_SMOTE = rus.fit_resample(X_train_SMOTE, y_train_SMOTE)

# Scale the training data
scaler_SMOTE = StandardScaler()
X_SMOTE_scaled = scaler_SMOTE.fit_transform(X_train_SMOTE)


# Create logistic regression object and fit on scaled SMOTE train
logi_mod2 = LogisticRegression(random_state=SEED)
trained_logistic_SMOTE = logi_mod2.fit(X_SMOTE_scaled, y_train_SMOTE)



- use classification report to get the information on model

In [17]:
from sklearn.metrics import classification_report

In [18]:
print("classification Report for ADA Booste Decision tree:")
print(classification_report(y_train, trained_ada_boost))

classification Report for ADA Booste Decision tree:


TypeError: Expected sequence or array-like, got <class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>